In [1]:
from langchain_community.llms import LlamaCpp
llm = LlamaCpp(model_path="/home/technoidentity/Downloads/gemma-2b.gguf")

llama_model_loader: loaded meta data with 19 key-value pairs and 164 tensors from /home/technoidentity/Downloads/gemma-2b.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = gemma
llama_model_loader: - kv   1:                               general.name str              = gemma-2b
llama_model_loader: - kv   2:                       gemma.context_length u32              = 8192
llama_model_loader: - kv   3:                          gemma.block_count u32              = 18
llama_model_loader: - kv   4:                     gemma.embedding_length u32              = 2048
llama_model_loader: - kv   5:                  gemma.feed_forward_length u32              = 16384
llama_model_loader: - kv   6:                 gemma.attention.head_count u32              = 8
llama_model_loader: - kv   7:              gemma.attention.head_coun

In [2]:
from pydantic import BaseModel

class MultiplicationToolSchema(BaseModel):
    operation: str
    number1: float
    number2: float

In [3]:
from langchain.tools import Tool

def multiplication_tool_function(input: dict) -> float:
    multiplication_tool_schema = MultiplicationToolSchema.parse_obj(input)
    return multiplication_tool_schema.number1 * multiplication_tool_schema.number2

multiplication_tool = Tool(
    name="multiplication",
    description="Multiplies two numbers.",
    func=multiplication_tool_function,
    input_schema=MultiplicationToolSchema.schema(),
)

In [4]:
fixed_prompt = "you are a powerful assistant, with knowledge on various things.But you don't have any knowledge on how to perform multiplication or find the product of any two numbers.You must always try to use one of the available tools to provide an answer for anything related to multiplication"

In [14]:
from typing import Dict,List
from langchain.schema import BaseMessage

class CustomMessageFormatter:
    def format_to_messages(self, input_data: Dict[str, str]) -> List[BaseMessage]:
        return [
            HumanMessage(content=f"{input_data['operation']} {input_data['number1']} {input_data['operation_symbol']} {input_data['number2']}"),
        ]


In [15]:
multiplication_prompt = "{{ operation }} {{ number1 }} {{ operation_symbol }} {{ number2 }}}"


In [16]:
from langchain.agents import create_tool_calling_agent

multiplication_agent = create_tool_calling_agent(
    llm=llm,
    tools=[multiplication_tool],
    prompt=multiplication_prompt,
    message_formatter=CustomMessageFormatter(),
)

AttributeError: 'str' object has no attribute 'input_variables'

In [ ]:
from langchain.agents import AgentExecutor

multiplication_agent_executor = AgentExecutor.from_agent_and_tools(
    agent=multiplication_agent,
    tools=[multiplication_tool],
    verbose=True,
)

In [ ]:
product = multiplication_agent_executor.run(
    input={
        "operation": "multiply",
        "number1": 5,
        "number2": 3,
    }
)
print(product) # Output: 15

In [20]:
from typing import *
from langchain.schema import BaseMessage, HumanMessage
from langchain.schema import BaseMessageFormatter

class CustomMessageFormatter(BaseMessageFormatter):
    def format_to_messages(self, input_data: Dict[str, float]) -> List[BaseMessage]:
        return [HumanMessage(content=f"{input_data['number1']} * {input_data['number2']}")]

# Create a multiplication tool
from langchain.tools import BaseTool

class MultiplyTool(BaseTool):
    name = "multiply"
    description = "Multiply two numbers"
    args_schema = {
        "type": "object",
        "properties": {
            "number1": {"type": "number"},
            "number2": {"type": "number"},
        },
        "required": ["number1", "number2"],
    }

    def _run(self, number1: float, number2: float) -> float:
        return number1 * number2

# Create a custom tool
custom_tool = MultiplyTool(name="custom_multiply")

# Create a custom message formatter
message_formatter = CustomMessageFormatter()

# Create a tool input
tool_input = {"number1": 3.14, "number2": 2.718}

# Format the tool input
formatted_input = message_formatter.format(tool_input)

# Pass the formatted input to the custom tool
output = custom_tool.run(formatted_input)

# Print the output
print(output)  # Output: 8.53472

ModuleNotFoundError: No module named 'langchain.schema.formatters'